In [1]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, GRU


# Define the path to your dataset
data_path = 'C:/Users/harsh/Desktop/wav'

# Get the list of genres from the directory names
genres = os.listdir(data_path)

# Initialize lists to hold features and labels
features = []
labels = []

# Iterate over each genre
for genre in genres:
    # Get the list of song files for this genre
    song_files = os.listdir(os.path.join(data_path, genre))
    
    # Iterate over each song file
    for song in song_files:
        # Load the song file using librosa
        y, sr = librosa.load(os.path.join(data_path, genre, song))
        
        # Extract features from the audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        
        # Append the mean of the features and the genre label to the respective lists
        features.append(np.mean(mfcc, axis=1))
        labels.append(genre)

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape the data for the CRNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dropout(0.25))
model.add(Dense(len(genres), activation='softmax'))


# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=42, validation_data=(X_test, y_test))






Epoch 1/100


20/20 [==============================] - 13s 143ms/step - loss: 2.2643 - accuracy: 0.1890 - val_loss: 2.1885 - val_accuracy: 0.1800
Epoch 2/100
20/20 [==============================] - 1s 28ms/step - loss: 2.1489 - accuracy: 0.2153 - val_loss: 2.1422 - val_accuracy: 0.2100
Epoch 3/100
20/20 [==============================] - 1s 28ms/step - loss: 2.0600 - accuracy: 0.2741 - val_loss: 2.0670 - val_accuracy: 0.2250
Epoch 4/100
20/20 [==============================] - 1s 28ms/step - loss: 2.0192 - accuracy: 0.2766 - val_loss: 2.0310 - val_accuracy: 0.2250
Epoch 5/100
20/20 [==============================] - 1s 28ms/step - loss: 1.9706 - accuracy: 0.3091 - val_loss: 1.9909 - val_accuracy: 0.2800
Epoch 6/100
20/20 [==============================] - 1s 27ms/step - loss: 1.9463 - accuracy: 0.3116 - val_loss: 1.9594 - val_accuracy: 0.2500
Epoch 7/100
20/20 [==============================] - 1s 28ms/step - loss: 1.8922 - accuracy: 0.3329 - val_loss: 1.9211 - val_accuracy: 0.290

Epoch 50/100
20/20 [==============================] - 0s 9ms/step - loss: 1.4803 - accuracy: 0.4819 - val_loss: 1.5565 - val_accuracy: 0.4050
Epoch 51/100
20/20 [==============================] - 0s 9ms/step - loss: 1.4537 - accuracy: 0.4869 - val_loss: 1.5426 - val_accuracy: 0.4300
Epoch 52/100
20/20 [==============================] - 0s 9ms/step - loss: 1.4808 - accuracy: 0.4681 - val_loss: 1.5221 - val_accuracy: 0.4550
Epoch 53/100
20/20 [==============================] - 1s 28ms/step - loss: 1.4621 - accuracy: 0.4944 - val_loss: 1.5490 - val_accuracy: 0.4350
Epoch 54/100
20/20 [==============================] - 1s 28ms/step - loss: 1.4510 - accuracy: 0.5006 - val_loss: 1.5503 - val_accuracy: 0.4450
Epoch 55/100
20/20 [==============================] - 1s 29ms/step - loss: 1.4228 - accuracy: 0.5006 - val_loss: 1.5713 - val_accuracy: 0.4350
Epoch 56/100
20/20 [==============================] - 1s 27ms/step - loss: 1.4450 - accuracy: 0.4869 - val_loss: 1.5762 - val_accuracy: 0.4350
Ep

In [2]:
# Make predictions on all instances in the test data
predictions = model.predict(X_test)

# Convert predictions to labels
predicted_labels = [np.argmax(prediction) for prediction in predictions]

print(f"Predicted Labels: {predicted_labels}")


7/7 [==============================] - 2s 9ms/step
Predicted Labels: [7, 7, 7, 2, 2, 8, 4, 2, 1, 6, 3, 6, 4, 8, 7, 2, 0, 4, 9, 4, 2, 3, 3, 3, 4, 6, 1, 4, 8, 3, 5, 7, 6, 5, 6, 6, 1, 1, 6, 5, 7, 5, 7, 4, 5, 5, 3, 7, 6, 8, 6, 1, 1, 0, 0, 8, 1, 8, 8, 7, 6, 6, 8, 1, 7, 3, 5, 1, 8, 5, 2, 1, 6, 1, 3, 5, 0, 7, 1, 9, 2, 5, 9, 8, 9, 3, 6, 5, 4, 4, 4, 4, 2, 1, 7, 0, 2, 8, 6, 4, 2, 4, 2, 6, 5, 7, 2, 5, 2, 0, 2, 1, 8, 9, 2, 3, 5, 7, 7, 2, 2, 2, 8, 5, 3, 2, 1, 9, 6, 7, 9, 9, 5, 6, 1, 7, 5, 2, 0, 2, 5, 2, 0, 2, 1, 8, 6, 8, 5, 7, 9, 4, 7, 7, 0, 6, 6, 5, 8, 9, 7, 0, 2, 6, 4, 6, 5, 4, 5, 1, 9, 9, 6, 9, 3, 7, 0, 3, 7, 8, 2, 1, 7, 4, 5, 4, 2, 5, 9, 8, 0, 2, 3, 2, 3, 9, 0, 9, 2, 3]


In [3]:
# Make predictions on all instances in the test data
predictions = model.predict(X_test)

# Convert predictions to labels
predicted_labels = [np.argmax(prediction) for prediction in predictions]

print(f"Predicted Labels: {predicted_labels}")


7/7 [==============================] - 0s 9ms/step
Predicted Labels: [7, 7, 7, 2, 2, 8, 4, 2, 1, 6, 3, 6, 4, 8, 7, 2, 0, 4, 9, 4, 2, 3, 3, 3, 4, 6, 1, 4, 8, 3, 5, 7, 6, 5, 6, 6, 1, 1, 6, 5, 7, 5, 7, 4, 5, 5, 3, 7, 6, 8, 6, 1, 1, 0, 0, 8, 1, 8, 8, 7, 6, 6, 8, 1, 7, 3, 5, 1, 8, 5, 2, 1, 6, 1, 3, 5, 0, 7, 1, 9, 2, 5, 9, 8, 9, 3, 6, 5, 4, 4, 4, 4, 2, 1, 7, 0, 2, 8, 6, 4, 2, 4, 2, 6, 5, 7, 2, 5, 2, 0, 2, 1, 8, 9, 2, 3, 5, 7, 7, 2, 2, 2, 8, 5, 3, 2, 1, 9, 6, 7, 9, 9, 5, 6, 1, 7, 5, 2, 0, 2, 5, 2, 0, 2, 1, 8, 6, 8, 5, 7, 9, 4, 7, 7, 0, 6, 6, 5, 8, 9, 7, 0, 2, 6, 4, 6, 5, 4, 5, 1, 9, 9, 6, 9, 3, 7, 0, 3, 7, 8, 2, 1, 7, 4, 5, 4, 2, 5, 9, 8, 0, 2, 3, 2, 3, 9, 0, 9, 2, 3]


In [4]:
from sklearn.metrics import classification_report

# Make predictions on all instances in the test data
predictions = model.predict(X_test)

# Convert predictions to labels
predicted_labels = [np.argmax(prediction) for prediction in predictions]

# Calculate precision, recall, F1-score, and accuracy
report = classification_report(y_test, predicted_labels, target_names=genres)

print(report)


7/7 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

       blues       0.46      0.29      0.35        21
   classical       0.58      0.92      0.71        12
     country       0.34      0.42      0.38        24
       disco       0.38      0.27      0.32        22
      hiphop       0.33      0.40      0.36        15
        jazz       0.62      0.56      0.59        27
       metal       0.61      0.78      0.68        18
         pop       0.67      0.84      0.74        19
      reggae       0.50      0.41      0.45        22
        rock       0.31      0.25      0.28        20

    accuracy                           0.49       200
   macro avg       0.48      0.51      0.49       200
weighted avg       0.48      0.49      0.48       200



In [5]:
# Combine the training and testing data
X_total = np.concatenate((X_train, X_test))
y_total = np.concatenate((y_train, y_test))

# Evaluate the model on the total data
total_loss, total_accuracy = model.evaluate(X_total, y_total)

print(f"Total Loss: {total_loss}")
print(f"Total Accuracy: {total_accuracy}")


32/32 [==============================] - 0s 8ms/step - loss: 1.1225 - accuracy: 0.6236
Total Loss: 1.122497797012329
Total Accuracy: 0.6236236095428467
